# Setup

In [1]:
%load_ext autoreload
%autoreload 2

import sys
from pathlib import Path

# Path('../backend/shuscribe').resolve()
sys.path.insert(0, str(Path('../backend').resolve()))

In [2]:
# CONSTANTS

USE_CACHED_RESPONSES = False 
# TODO: KEEP CHANGING THIS TO GET THE NEXT CHAPTER, STARTING FROM 1
CHAPTER_INDEX = 1

In [3]:
# Import necessary modules
import asyncio
from dotenv import load_dotenv
import os
import json
from shuscribe.services.llm.session import LLMSession
from shuscribe.services.llm.providers.provider import (
    Message, GenerationConfig
)
from IPython.display import Markdown

load_dotenv()
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
ANTHROPIC_API_KEY = os.environ["ANTHROPIC_API_KEY"]
GEMINI_API_KEY = os.environ["GEMINI_API_KEY"]

TEST_MODELS ={
    "openai": "gpt-4o-mini",
    "anthropic": "claude-3-5-haiku-20241022",
    "gemini": "gemini-2.0-flash-001"
}

TEST_THINKING_MODELS = {
    "openai": "o3-mini-2025-01-31",
    "anthropic": "claude-3-7-sonnet-20250219",
    "gemini": "gemini-2.0-flash-thinking-exp"
}

STORY_DIR = Path("../backend/tests/resources/pokemon_amber")

# Helper function to run async code in notebook
async def run_async(coro):
    return await coro

In [4]:
# Streaming response
from typing import Type
from shuscribe.schemas.base import BaseOutputSchema
from shuscribe.schemas.llm import ThinkingConfig
from shuscribe.schemas.streaming import StreamChunk
from shuscribe.services.llm.streaming import StreamStatus


async def stream(
    provider_name: str, 
    model: str,
    messages: list[Message], 
    response_schema: Type[BaseOutputSchema] | None = None, 
    max_tokens: int | None = None,
    temp: float | None = None,
    thinking_config: ThinkingConfig | None = None,
    config: GenerationConfig | None = None
    ) -> StreamChunk | None:
    async with LLMSession.session_scope() as session:
        # Create a streaming config
        if not config:
            config = GenerationConfig(
                temperature=temp or 0.7,
                response_schema=response_schema if response_schema else None,
                max_output_tokens=max_tokens,
                thinking_config=thinking_config
            )
        
        print(f"{model if model else config.model}:")

        async for chunk in session.generate_stream(
            messages=messages,
            provider_name=provider_name,
            model=model,
            config=config
        ):
            print(chunk.text, end="", flush=True)

    if chunk:
        if chunk.status in (StreamStatus.COMPLETE, StreamStatus.ERROR):
            return chunk
    else:
        return None


In [5]:
import yaml

from shuscribe.schemas.pipeline import Chapter, StoryMetadata

with open(STORY_DIR / "story" / "_meta.yaml", "r") as f:
    meta = yaml.safe_load(f)
    STORY_METADATA = StoryMetadata(
        title=meta.get('story_title'),
        description=meta.get('story_description'),
        genres=meta.get('genres'),
        additional_tags=meta.get('additional_tags')
    )

CHAPTERS = []
for chapter in meta.get('chapters'):
    with open(STORY_DIR / "story" / chapter, "r") as f:
        chapter_id = chapter.split('.')[0]
        try:
            chapter_content = yaml.safe_load(f)
            CHAPTERS.append(Chapter(id=chapter_id, title=chapter_content.get('title'), content=chapter_content.get('content')))
        except Exception:
            continue

CHAPTERS

[Chapter(title='[Chapter 1] Truck-kun Strikes Again', id=1, content='\'Fuck,\' I thought, hunched over my phone in the fading afternoon light. My fingers hovered over the screen as if I could somehow intimidate the numbers into change.\nMy Gyarados—level 40, carefully trained, survivor of multiple gym battles and approximately eight thousand random Pidgey and Mankey encounters—was down to a sliver of health. Across the battlefield, a level 37 Vulpix stared back with pixels that somehow managed to look smug. It had probably practiced that look in the mirror.\nI\'d already lost two good Pokemon this past hour. My box was only down to shitty Pokemon. I couldn\'t lose another. Not now.\nThe Pokemon Mansion sprawled across my phone screen in all its 8-bit glory, a maze of broken tiles and forgotten experiments. Outside my dorm window, campus life continued its usual Sunday afternoon rhythm—distant voices, someone practicing guitar badly, and the occasional burst of laughter. None were aware

# Summarization Pipeline

## Chapter [X]

In [6]:

from shuscribe.schemas.wikigen.entity import TempEntityDBRepresentation
from shuscribe.schemas.wikigen.entity import TempEntityDB
from shuscribe.schemas.wikigen.wiki import WikiPage
from shuscribe.schemas.wikigen.summary import ChapterSummary
from shuscribe.services.llm.pipeline.pipelines.wikigen import CurrentState

CURRENT_STATE = CurrentState()

if CHAPTER_INDEX > 0:
    with open(STORY_DIR / f"{CHAPTER_INDEX-1}out" / "comprehensive_wiki.md", "r") as f:
        COMPREHENSIVE_WIKI = WikiPage.from_wiki_content("Comprehensive Wiki Page", f.read())
    CURRENT_STATE.summary_so_far = COMPREHENSIVE_WIKI

    prev_chapter_index = CHAPTER_INDEX - 1
    for i in range(prev_chapter_index, -1, -1):
        summary_path = STORY_DIR / f"{i}out" / "chapter_summary.md"
        with open(summary_path, "r") as f:
            summary_str = f.read()
            success = CURRENT_STATE.insert_old_chap_summary(ChapterSummary.from_chapter_summary(
                i, 
                summary_str
            ))


ENTITY_DB = TempEntityDB()

if CHAPTER_INDEX > 0:
    with open(STORY_DIR / f"{CHAPTER_INDEX-1}out" / "entity_db.json", "r") as f:
        entities_db = TempEntityDBRepresentation.model_validate_json(f.read())

    ENTITY_DB.entities_db = entities_db

!!!!!!!!!!!!megablocks not available, using torch.matmul instead
<All keys matched successfully>


In [7]:
CURRENT_STATE

CurrentState(summary_so_far=WikiPage(title='Comprehensive Wiki Page', content='\n## Overview\n\nThis story follows a protagonist, an avid Pokémon player from another world, who is reincarnated into the [[Pokemon World (Ambertwo)]] as [[Amber (Protagonist)]], a clone apparently created by [[Dr. Fuji]]. Awakening during [[Mewtwo]]\'s destructive escape from the [[New Island Laboratory]], she must navigate the immediate dangers and the implications of her new existence.\n\n## Death and Reincarnation\n\n### The Gamer\'s Last Stand\n\nIn her previous life, the protagonist was deeply engrossed in a Pokémon emulator game on her phone, featuring perma-death rules. During a critical battle in the Pokémon Mansion where her Gyarados faced a Vulpix, she risked losing significant progress. Simultaneously, a Discord notification alerted her to a limited-time Shadow Mewtwo raid in Pokémon GO happening nearby at her university campus library.\n\nChoosing the raid over her emulator battle, she saved he

### Chapter Summary

In [8]:
from shuscribe.schemas.wikigen.summary import ChapterSummary
from shuscribe.services.llm.prompts import templates

templates.chapter.summary.reload()

summary_messages: list[Message] = templates.chapter.summary.format( 
    current_chapter=CHAPTERS[CHAPTER_INDEX],
    story_metadata=STORY_METADATA,
    summary_so_far=CURRENT_STATE.summary_so_far,
    recent_summaries=CURRENT_STATE.recent_chap_summaries,
)

# print(summary_messages[-1].content)

chapter_summary = None

if USE_CACHED_RESPONSES:
    # if exists, load from file
    if (STORY_DIR / f"{CHAPTER_INDEX}out" / "chapter_summary.md").exists():
        with open(STORY_DIR / f"{CHAPTER_INDEX}out" / "chapter_summary.md", "r") as f:
            chapter_summary = ChapterSummary.from_chapter_summary(CHAPTER_INDEX, f.read())

# if we don't have a cached response, generate one
if not chapter_summary:
    summary_response: StreamChunk = await run_async(stream(
        "", 
        "", 
        summary_messages, 
        config=templates.chapter.summary.default_config
        ))
    print(summary_response.usage)
    chapter_summary = ChapterSummary.from_chapter_summary(CHAPTER_INDEX, summary_response.accumulated_text)

os.makedirs(STORY_DIR / f"{CHAPTER_INDEX}out", exist_ok=True)
with open(STORY_DIR / f"{CHAPTER_INDEX}out" / "chapter_summary.md", "w") as f:
    f.write(chapter_summary.raw_summary)

CURRENT_STATE.add_new_chap_summary(chapter_summary)

gemini-2.0-flash-001:
<|STARTOFSUMMARY|>

## Lingering Trauma and Existential Dread

*   The protagonist, now referred to as "Amber-Two," struggles with sleep, haunted by visions of psychic energy and the memory of Mewtwo's awakening. [!CHARACTER]
    *   She experiences a burning sensation in her lungs, a physical reminder of the cloning process. [!WORLD]
*   Amber-Two grapples with her new, ten-year-old body, feeling disconnected from it and burdened by adult thoughts and memories. [!CHARACTER] [!THEME]
    *   She reflects on the irony of Dr. Fuji's actions, creating her anew only for her to be inhabited by someone who died senselessly. [!CHARACTER]
*   She feels like an imposter, occupying a body that belongs to the original Amber, whose grave exists somewhere. [!CHARACTER] [!THEME]
*   Amber-Two contemplates the ignorance of tourists visiting Cinnabar Island, unaware of the hidden laboratory and its dangerous experiments. [!WORLD]
*   She recognizes the parallel between herself an

Database module not implemented. Skipping save.


prompt_tokens=4865 completion_tokens=833


False

### Extract Entities

In [9]:
CURRENT_STATE.add_new_chap_summary(chapter_summary)
# if current_state.chapter_summary:
#     print(current_state.chapter_summary.to_prompt())

False

In [18]:
print(CURRENT_STATE.recent_chap_summaries[1].summary)

## Lingering Trauma and Existential Dread
*   The protagonist, now referred to as "Amber-Two," struggles with sleep, haunted by visions of psychic energy and the memory of Mewtwo's awakening.
    *   She experiences a burning sensation in her lungs, a physical reminder of the cloning process.
*   Amber-Two grapples with her new, ten-year-old body, feeling disconnected from it and burdened by adult thoughts and memories.
    *   She reflects on the irony of Dr. Fuji's actions, creating her anew only for her to be inhabited by someone who died senselessly.
*   She feels like an imposter, occupying a body that belongs to the original Amber, whose grave exists somewhere.
*   Amber-Two contemplates the ignorance of tourists visiting Cinnabar Island, unaware of the hidden laboratory and its dangerous experiments.
*   She recognizes the parallel between herself and Mewtwo, both unnatural beings grappling with their existence.
    *   However, she feels she lacks Mewtwo's purpose, seeing herse

In [18]:
from shuscribe.schemas.wikigen.entity import ExtractEntitiesOutSchema


templates.entity.extract.reload()
extract_messages: list[Message] = templates.entity.extract.format( 
    current_chapter=CHAPTERS[CHAPTER_INDEX],
    # current_chapter=Chapter(id=1, title="Chapter 1", content="This is a test chapter."),
    story_metadata=STORY_METADATA,
    chapter_summary=chapter_summary,
    summary_so_far=CURRENT_STATE.summary_so_far,
    recent_summaries=CURRENT_STATE.recent_chap_summaries,
)

# print(extract_messages[-1].content)

extracted_entities = None
if USE_CACHED_RESPONSES:
    # if exists, load from file
    if (STORY_DIR / f"{CHAPTER_INDEX}out" / "extract_entities.json").exists():
        with open(STORY_DIR / f"{CHAPTER_INDEX}out" / "extract_entities.json", "r") as f:
            extracted_entities = StreamChunk.model_validate_json(f.read())
            
# if we don't have a cached response, generate one
if not extracted_entities:  # type: ignore
    extract_response: StreamChunk = await run_async(stream(
        "", 
        "", 
        extract_messages, 
        config=templates.entity.extract.default_config
        ))
    print(extract_response.usage)
    extracted_entities = ExtractEntitiesOutSchema.model_validate_json(extract_response.accumulated_text)
    
os.makedirs(STORY_DIR / f"{CHAPTER_INDEX}out", exist_ok=True)
with open(STORY_DIR / f"{CHAPTER_INDEX}out" / "extract_entities.json", "w") as f:
    f.write(extract_response.model_dump_json(indent=2))

extracted_entities: ExtractEntitiesOutSchema


gemini-2.5-pro-exp-03-25:
{
  "entities": [
    {
      "description": "The main character, a female Pokemon gamer who dies in her original world and is reincarnated into the Pokemon world as a clone of Dr. Fuji's deceased daughter, Amber.",
      "narrative_role": "Protagonist",
      "significance_level": "Central",
      "entity_type": "Character",
      "identifier": "Protagonist (Ambertwo)",
      "aliases": ["I", "Amber", "Fuji's legendary clone", "AlexaTheGreat"],
      "related_entities": ["Dr. Fuji", "Amber (Original)", "Mewtwo", "Reincarnation/Isekai", "Dr. Fuji's Laboratory", "Pokemon Emulator Game", "Protagonist's Death"]
    },
    {
      "description": "A brilliant scientist who successfully cloned his deceased daughter, resulting in the protagonist's new body. He is also heavily implied to be involved in Mewtwo's creation. He rescues the protagonist from the destruction caused by Mewtwo.",
      "narrative_role": "Guardian/Creator Figure",
      "significance_level": "C

Database module not implemented. Skipping save.


prompt_tokens=4945 completion_tokens=2914


In [20]:
CURRENT_STATE.entities = extracted_entities

### Search for Entities


In [21]:
from shuscribe.schemas.wikigen.entity import EntitySigLvl, TempEntityRecord

if CURRENT_STATE.entities:
    ent_list = CURRENT_STATE.entities.filter_entities(EntitySigLvl.RELEVANT)

existing_entities: set[TempEntityRecord] = set()

for entity in ent_list:
    print(entity.to_upsert_dict(ent_list))
    for result in ENTITY_DB.search(json.dumps(entity.to_upsert_dict(ent_list), indent=2)):
        existing_entities.add(result[0])

print(len(existing_entities))

for entity in existing_entities:
    print(entity)


{'identifier': 'Protagonist (Ambertwo)', 'aliases': ['I', 'Amber', "Fuji's legendary clone", 'AlexaTheGreat'], 'entity_type': 'Character', 'significance_level': 'Central', 'narrative_role': 'Protagonist', 'description': "The main character, a female Pokemon gamer who dies in her original world and is reincarnated into the Pokemon world as a clone of Dr. Fuji's deceased daughter, Amber.", 'related_entities': ['Dr. Fuji', 'Mewtwo', 'Reincarnation/Isekai', "Dr. Fuji's Laboratory", 'Amber (Original)', "Protagonist's Death", 'Pokemon Emulator Game']}
{'identifier': 'Dr. Fuji', 'aliases': ['Giant middle-aged man'], 'entity_type': 'Character', 'significance_level': 'Central', 'narrative_role': 'Guardian/Creator Figure', 'description': "A brilliant scientist who successfully cloned his deceased daughter, resulting in the protagonist's new body. He is also heavily implied to be involved in Mewtwo's creation. He rescues the protagonist from the destruction caused by Mewtwo.", 'related_entities':

### Upsert Entities and Relationships

In [24]:
from shuscribe.schemas.provider import LLMUsage
from shuscribe.schemas.wikigen.entity import UpsertEntitiesOutSchema
from shuscribe.schemas.wikigen.entity import EntitySigLvl

upsert_entities = UpsertEntitiesOutSchema(entities=[])

if USE_CACHED_RESPONSES:
    # if exists, load from file
    if (STORY_DIR / f"{CHAPTER_INDEX}out" / "upsert_entities.json").exists():
        with open(STORY_DIR / f"{CHAPTER_INDEX}out" / "upsert_entities.json", "r") as f:
            upsert_entities = UpsertEntitiesOutSchema.model_validate_json(f.read())

if len(upsert_entities.entities) == 0:
    upsert_entities = UpsertEntitiesOutSchema(entities=[])
    total_usage = LLMUsage(prompt_tokens=0, completion_tokens=0)
    templates.entity.upsert.reload()        

    for batch in extracted_entities.batch_for_upsert(EntitySigLvl.RELEVANT, chunk_size=5):
        upsert_messages: list[Message] = templates.entity.upsert.format( 
            current_chapter=CHAPTERS[CHAPTER_INDEX],
            entity_batch=batch,
            story_metadata=STORY_METADATA,
            chapter_summary=CURRENT_STATE.chapter_summary,
            
            existing_entities=[entity for entity in existing_entities],
            summary_so_far=CURRENT_STATE.summary_so_far,
            recent_summaries=CURRENT_STATE.recent_chap_summaries,
        )
        upsert_response = await run_async(stream(
            "", 
            "", 
            upsert_messages, 
            config=templates.entity.upsert.default_config
            ))
        upsert_entities.entities.extend(UpsertEntitiesOutSchema.model_validate_json(upsert_response.accumulated_text).entities)
        total_usage.prompt_tokens += upsert_response.usage.prompt_tokens
        total_usage.completion_tokens += upsert_response.usage.completion_tokens
        print(upsert_response.usage)

os.makedirs(STORY_DIR / f"{CHAPTER_INDEX}out", exist_ok=True)
with open(STORY_DIR / f"{CHAPTER_INDEX}out" / "upsert_entities.json", "w") as f:
    f.write(upsert_entities.model_dump_json(indent=2))


gemini-2.0-flash-001:
{
  "entities": [
    {
      "identifier": "Protagonist (Ambertwo)",
      "old_identifier": null,
      "detailed_description": "*   The main character of the story.\n*   A female Pokémon gamer from another world who died after being hit by a truck.\n*   Reincarnated into the Pokémon world as a clone of Dr. Fuji's deceased daughter, Amber.\n*   Initially disoriented and confused upon awakening in a cloning tank.\n*   Expresses concern for Dr. Fuji's safety and recognizes the danger of the situation.\n*   Currently in a child's body, implied to be around the same age as Amber was when she died.\n*   Goes by the online handle \"AlexaTheGreat\".",
      "narrative_role": "Protagonist",
      "facts": [
        {
          "fact": "The protagonist was a dedicated Pokemon player in her previous life.",
          "type": "Explicit"
        },
        {
          "fact": "The protagonist's online handle is AlexaTheGreat.",
          "type": "Explicit"
        }
      ]

Database module not implemented. Skipping save.


prompt_tokens=6545 completion_tokens=2841
gemini-2.0-flash-001:
{
  "entities": [
    {
      "identifier": "Amber (Original)",
      "old_identifier": "Amber (Original)",
      "detailed_description": "*   Dr. Fuji's deceased daughter.\n*   Her death is Dr. Fuji's primary motivation for his cloning experiments, particularly the creation of Mewtwo and the protagonist.\n*   The protagonist is cloned from her DNA.\n*   Dr. Fuji is desperate to revive her and \"fix\" their family.",
      "narrative_role": "Motivator/Original Identity",
      "facts": [],
      "removed_facts": [],
      "entity_types": [
        "Character"
      ],
      "aliases": [
        "Amber"
      ],
      "related_entities": [
        {
          "target_entity_id": "Protagonist (Ambertwo)",
          "description": "The protagonist is a clone of Amber (Original).",
          "relationship_type": "Causal",
          "is_bidirectional": false
        },
        {
          "target_entity_id": "Dr. Fuji",
       

Database module not implemented. Skipping save.


prompt_tokens=6324 completion_tokens=1920
gemini-2.0-flash-001:
{
  "entities": [
    {
      "identifier": "Primal Terror/Instinct",
      "old_identifier": "Primal Terror/Instinct",
      "detailed_description": "*   A deep, instinctual fear experienced by the protagonist upon making eye contact with Mewtwo.\n*   Overrode her gamer knowledge and rational thought.\n*   Triggered by Mewtwo's raw, primal power and existence outside the natural order.\n*   Represents the protagonist's vulnerability in her new body and the overwhelming nature of the Pokémon world's reality.",
      "narrative_role": "Character Emotion/Thematic Element",
      "facts": [],
      "removed_facts": [],
      "entity_types": [
        "Concept",
        "Theme"
      ],
      "aliases": [
        "Pokemon instinct"
      ],
      "related_entities": [
        {
          "target_entity_id": "Protagonist (Ambertwo)",
          "description": "The protagonist experiences primal terror.",
          "relationship_

Database module not implemented. Skipping save.


prompt_tokens=6283 completion_tokens=1503
gemini-2.0-flash-001:
{
  "entities": [
    {
      "identifier": "Pokemon World (Pokemon: Ambertwo)",
      "old_identifier": "New World/Pokemon World",
      "detailed_description": "*   A fictional world based on the Pokemon franchise, now the primary setting for the protagonist's reincarnation.\n*   Contains locations like Dr. Fuji's Laboratory and potentially others from the Pokemon games/anime/manga.\n*   Operates under the rules and concepts of the Pokemon universe, including Pokemon, battles, and various technologies.\n*   Currently experiencing a period of chaos due to Mewtwo's escape and the destruction of the laboratory.\n*   The protagonist is now a part of this world, inhabiting a cloned body.",
      "narrative_role": "Primary Setting",
      "facts": [],
      "removed_facts": [],
      "entity_types": [
        "Location",
        "Concept"
      ],
      "aliases": [
        "Pokemon World",
        "New World"
      ],
      "

Database module not implemented. Skipping save.


prompt_tokens=5779 completion_tokens=541


### Update Entity DB

In [25]:
ENTITY_DB.upsert(upsert_entities.entities)
ENTITY_DB.size()

16

In [27]:
ENTITY_DB

In [28]:
with open(STORY_DIR / f"{CHAPTER_INDEX}out" / "entity_db.json", "w") as f:
    f.write(ENTITY_DB.entities_db.model_dump_json(indent=2))
    
with open(STORY_DIR / f"{CHAPTER_INDEX}out" / "~entity_db.json", "w") as f:
    f.write(ENTITY_DB.entities_db.model_dump_json(indent=2, exclude={"entities": {key: {"embedding"} for key in ENTITY_DB.entities_db.entities}}))


### Story So Far Summary

In [ ]:

from shuscribe.schemas.wikigen.wiki import WikiPage


templates.story.comprehensive_wiki.reload()
if not CURRENT_STATE.chapter_summary:
    raise ValueError("Chapter summary is required to generate comprehensive wiki")

comprehensive_wiki_messages: list[Message] = templates.story.comprehensive_wiki.format( 
    current_chapter=CHAPTERS[CHAPTER_INDEX],
    chapter_summary=CURRENT_STATE.chapter_summary,
    
    story_metadata=STORY_METADATA,
    summary_so_far=CURRENT_STATE.summary_so_far,
    recent_summaries=CURRENT_STATE.recent_chap_summaries,
)

# print(comprehensive_summary_messages[-1].content)
comprehensive_wiki = None
if USE_CACHED_RESPONSES:
    # if exists, load from file
    if (STORY_DIR / f"{CHAPTER_INDEX}out" / "comprehensive_wiki.md").exists():
        with open(STORY_DIR / f"{CHAPTER_INDEX}out" / "comprehensive_wiki.md", "r") as f:
            comprehensive_wiki = WikiPage.from_wiki_content("Comprehensive Wiki Page", f.read())

if not comprehensive_wiki:
    comprehensive_wiki_response: StreamChunk = await run_async(stream(
        "", 
        "", 
        comprehensive_wiki_messages, 
        config=templates.story.comprehensive_wiki.default_config
        ))
    print(comprehensive_wiki_response.usage)
    comprehensive_wiki = WikiPage.from_wiki_content("Comprehensive Wiki Page", comprehensive_wiki_response.accumulated_text)

os.makedirs(STORY_DIR / f"{CHAPTER_INDEX}out", exist_ok=True)
with open(STORY_DIR / f"{CHAPTER_INDEX}out" / "comprehensive_wiki.md", "w") as f:
    f.write(comprehensive_wiki.raw_content)


gemini-2.5-pro-exp-03-25:
<|STARTWIKI|>
## Overview

This story follows a protagonist, an avid Pokémon player from another world, who is reincarnated into the [[Pokemon World (Ambertwo)]] as [[Amber (Protagonist)]], a clone apparently created by [[Dr. Fuji]]. Awakening during [[Mewtwo]]'s destructive escape from the [[New Island Laboratory]], she must navigate the immediate dangers and the implications of her new existence.

## Death and Reincarnation

### The Gamer's Last Stand

In her previous life, the protagonist was deeply engrossed in a Pokémon emulator game on her phone, featuring perma-death rules. During a critical battle in the Pokémon Mansion where her Gyarados faced a Vulpix, she risked losing significant progress. Simultaneously, a Discord notification alerted her to a limited-time Shadow Mewtwo raid in Pokémon GO happening nearby at her university campus library.

Choosing the raid over her emulator battle, she saved her game state and rushed towards the library. Distract

Database module not implemented. Skipping save.


prompt_tokens=5191 completion_tokens=881


In [30]:
# display markdown
Markdown(comprehensive_wiki.content)



## Overview

This story follows a protagonist, an avid Pokémon player from another world, who is reincarnated into the [[Pokemon World (Ambertwo)]] as [[Amber (Protagonist)]], a clone apparently created by [[Dr. Fuji]]. Awakening during [[Mewtwo]]'s destructive escape from the [[New Island Laboratory]], she must navigate the immediate dangers and the implications of her new existence.

## Death and Reincarnation

### The Gamer's Last Stand

In her previous life, the protagonist was deeply engrossed in a Pokémon emulator game on her phone, featuring perma-death rules. During a critical battle in the Pokémon Mansion where her Gyarados faced a Vulpix, she risked losing significant progress. Simultaneously, a Discord notification alerted her to a limited-time Shadow Mewtwo raid in Pokémon GO happening nearby at her university campus library.

Choosing the raid over her emulator battle, she saved her game state and rushed towards the library. Distracted by her phone and the urgency of the raid timer, she ignored traffic signals while crossing a street. She was struck and killed by a vehicle, her last thought being about her Gyarados left paused in the emulator.

### Rebirth in Amber

The protagonist's consciousness returned within a containment tank filled with viscous, amber-colored fluid located in the [[New Island Laboratory]]. She experienced muffled sounds, distorted vision, and the sensation of being suspended in a small, unfamiliar body that didn't seem to need air. The tank eventually fractured and shattered amidst tremors and a powerful, psychic roar.

## [[Mewtwo]]'s Escape and the Awakening of [[Amber (Protagonist)]]

### Encounter with [[Dr. Fuji]]

As the tank broke, the protagonist was caught by [[Dr. Fuji]], who was overcome with emotion, calling her "Amber" and expressing joy that she was alive. He wrapped her small form in his lab coat, which carried scents of antiseptic, coffee, and earth. The protagonist attempted to speak, realizing her voice was now that of a young girl. Seeing [[Dr. Fuji]]'s tears and hearing him call her Amber, combined with the surrounding chaos, led her to understand she inhabited the cloned body of [[Dr. Fuji]]'s deceased daughter.

### [[Mewtwo]]'s Rampage

Amidst the destruction, the protagonist saw [[Mewtwo]], terrifyingly real and immensely powerful, breaking free from its own containment. A brief moment of eye contact between [[Amber (Protagonist)]] and [[Mewtwo]] occurred, filling her with primal fear before it was broken by a security guard's Arcanine attacking [[Mewtwo]] with flames. Recognizing the catastrophic situation depicted in Pokémon lore surrounding [[Mewtwo]]'s creation, [[Amber (Protagonist)]] urged [[Dr. Fuji]] to flee.

## Escape from [[New Island Laboratory]]

### Navigating the Chaos

[[Dr. Fuji]], despite his emotional state, scooped up [[Amber (Protagonist)]] and began navigating the collapsing laboratory. The facility was filled with sparking equipment, shattered tanks, and steam. They witnessed other experimental Pokémon variants glitching in and out of reality. [[Mewtwo]], after being hit by the Arcanine again, blasted a hole through the ceiling to escape. [[Dr. Fuji]] shielded [[Amber (Protagonist)]] with his own body from the falling debris, sustaining injuries including cuts and bruises across his back.

### The Mansion Above

Emerging from the destroyed laboratory level, they entered corridors resembling a wealthy estate, suggesting the lab was hidden beneath or within a mansion. Panicked scientists and security personnel filled the hallways. Fragments of overheard conversations mentioned a security breach, containment failures in "Enhancement Lab Three," the overloading of an "evolution acceleration chamber," and referred to [[Amber (Protagonist)]] as "[[Fuji's legendary clone|Fuji's legendary clone]]".

### Uncertain Future

As they escaped the burning facility into the night, [[Dr. Fuji]], though injured, held [[Amber (Protagonist)]] close. He spoke of fixing their family and life, expressing a desperate hope for the future despite the disaster unfolding around them.

